In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import networkx as nx
from datetime import datetime
import matplotlib.pyplot as plt

import pynwb
from pynwb import register_class, load_namespaces
from pynwb.spec import NWBNamespaceBuilder, NWBGroupSpec, NWBDatasetSpec, NWBAttributeSpec, NWBLinkSpec
from pynwb.file import MultiContainerInterface, NWBContainer, NWBDataInterface
from hdmf.utils import docval

### Create the spec for franklab_apparatus
Instead of writing the spec from scratch, use the PyNWB helper functions for creating the spec.

In [3]:
name = 'franklab_apparatus'
ns_path = name + ".namespace.yaml"
ext_source = name + ".extensions.yaml"

help_attr = NWBAttributeSpec(name='help', doc='help doc', dtype='text', value='help value')

# Abstract represention for any kind of node in the topological graph
# A node must have a name
node = NWBGroupSpec(neurodata_type_def='Node',
                    neurodata_type_inc='NWBDataInterface',
                    doc='nodes in the graph',
                    quantity='+',
                    attributes=[NWBAttributeSpec('name', 'the name of this node', 'text'), 
                                help_attr])

# An abstract representation for edges between nodes in the graph
# An edge's only dataset is the name of the two nodes that the edge connects
edge = NWBGroupSpec(neurodata_type_def='Edge',
                    neurodata_type_inc='NWBDataInterface',
                    doc='edges in the graph',
                    quantity='*',
                    datasets=[NWBDatasetSpec(doc='names of the nodes this edge connects',
                                  name='edge_nodes',
                                  dtype='text',
                                  dims=['first_node_name|second_node_name'],
                                  shape=[2])],
                    attributes=[help_attr])

# A node that represents a single 2D point in space (e.g. reward well)
point_node = NWBGroupSpec(neurodata_type_def='PointNode',
                    neurodata_type_inc='Node',
                    doc='node representing a point in 2D space',
                    quantity='*',
                    datasets=[NWBDatasetSpec(doc='x/y coordinate of this 2D point',
                                  name='coords',
                                  dtype='float',
                                  dims=['num_coords', 'x_vals|y_vals'],
                                  shape=[1, 2])],
                    attributes=[help_attr])

# A node that represents a linear segement in 2D space, defined by its start and end points
# (e.g. a single arm of W-track maze)
segment_node = NWBGroupSpec(neurodata_type_def='SegmentNode',
                    neurodata_type_inc='Node',
                    doc='node representing a 2D linear segment defined by its start and end points',
                    quantity='*',
                    datasets=[NWBDatasetSpec(doc='x/y coordinates of the start and end points of this segment',
                                  name='coords',
                                  dtype='float',
                                  dims=['num_coords', 'x_vals|y_vals'],
                                  shape=[2, 2])],
                    attributes=[help_attr])

# A node that represents a polygon area defined by its external vertices and other control points (e.g. doors),
# and optionally, any interior points (e.g. interior wells, objects)
polygon_node = NWBGroupSpec(neurodata_type_def='PolygonNode',
                    neurodata_type_inc='Node',
                    doc='node representing a 2D polygon area',
                    quantity='*',
                    datasets=[NWBDatasetSpec(doc='x/y coordinates of the exterior points of this polygon',
                                  name='coords',
                                  dtype='float',
                                  dims=['num_coords', 'x_vals|y_vals'],
                                  shape=['null', 2]),
                              NWBDatasetSpec(doc='x/y coordinates of interior points inside this polygon',
                                  name='interior_coords',
                                  dtype='float',
                                  quantity='*',
                                  dims=['num_coords', 'x_vals|y_vals'],
                                  shape=['null', 2])],
                    attributes=[help_attr])

# A topological graph consisting of nodes and edges
apparatus = NWBGroupSpec(neurodata_type_def='Apparatus',
                         neurodata_type_inc='NWBDataInterface',
                         doc='a graph of nodes and edges', 
                         groups=[node, edge],
                         attributes=[NWBAttributeSpec(name='name', doc='the name of this apparatus', dtype='text'),
                                     help_attr])

# Task - description of a task and link to its associated apparatus
task = NWBGroupSpec(neurodata_type_def='Task',
                    neurodata_type_inc='NWBDataInterface',
                    doc='a behavioral task executed on a particular apparatus',
                    links=[NWBLinkSpec(name='apparatus', doc='the apparatus on which this task was performed', target_type='Apparatus')],
                    attributes=[NWBAttributeSpec(name='name', doc='the name of this task', dtype='text'),
                                NWBAttributeSpec(name='description', doc='description of this task', dtype='text'),
                                help_attr])


ns_builder = NWBNamespaceBuilder(name + ' extensions', name)
ns_builder.add_spec(ext_source, apparatus)
ns_builder.add_spec(ext_source, task)
ns_builder.add_spec(ext_source, point_node)
ns_builder.add_spec(ext_source, segment_node)
ns_builder.add_spec(ext_source, polygon_node)
ns_builder.export(ns_path)

ValueError: ('Cannot give specific name to something that can ', "exist multiple times: name='interior_coords', quantity='*'")